# Model Prediction

Three experiments:
- using Logisitc Regression
- using Random Decision Trees
- using a Bagging Classifier with a Random Decision Tree being the estimator

**Import Statements**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from scipy import stats
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [2]:
df = pd.read_csv("clean_data.csv")
df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis="columns")
df

,url,protocol,www_present,sub_domain,domain,top_domain,dir,file,path,fragment,...,n_hyphens,n_underscore,n_slash,n_questionmrk,n_equals,n_at,n_and,n_exclamation,url_length,domain_name_correct
0,https://ghfdc.knuodwq.cn/,1,0,1,1,1,0,0,1,0,...,0,0,3,0,0,0,0,0,25,1
1,https://rakutenluyaw.ouxawer-p.net/,1,0,1,1,1,0,0,1,0,...,1,0,3,0,0,0,0,0,35,1
2,pulsagratiss-1010.000webhostapp.com,0,0,1,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,35,1
3,idjvn.com,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,9,0
4,http://www.vmveg.com//vendor/phpunit/phpunit/s...,1,1,1,1,1,0,1,1,0,...,0,0,10,1,0,0,0,0,74,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98567,interceder.net/topic/Guy-Turcotte,0,0,0,1,1,1,1,1,0,...,1,0,2,0,0,0,0,0,33,1
98568,ifes.org/Divisions/List-Projects.aspx,0,0,0,1,1,1,1,1,0,...,1,0,2,0,0,0,0,0,37,0
98569,en.wikipedia.org/wiki/Jeffrey_Lynn,0,0,1,1,1,1,1,1,0,...,0,1,2,0,0,0,0,0,34,1
98570,ccdowney.com/bible-college/,0,0,0,1,1,1,0,1,0,...,1,0,2,0,0,0,0,0,27,1


**Determimg the number of outliers and removing them**

In [3]:
# determimg the number of outliers
clf = IsolationForest(contamination=0.005)
X = df.iloc[:, 1:].dropna()
y_pred = clf.fit_predict(X)
outliers = np.where(y_pred==-1)
outliers = outliers[0]
print(outliers)
print('Number of outliers:', len(outliers))
print(df.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


[  347   447   448   571   601   610   675   686   703   724  1012  1066
  1176  1239  1245  1468  1529  1643  1734  1940  1981  2006  2063  2095
  2128  2208  2210  2379  2595  2636  2678  2785  3040  3047  3081  3280
  3445  3470  3531  3589  3779  3817  3863  3917  4039  4482  4733  4773
  4784  5012  5092  5097  5164  5318  5360  5416  5751  5950  6006  6009
  6070  6188  6340  6345  6469  6499  6545  6907  6924  6956  6994  7379
  7541  7603  7723  7885  7996  8044  8046  8163  8362  8413  8480  8538
  8669  8775  8795  8823  8856  8860  9029  9115  9513  9516  9843 10172
 10196 10560 10567 10759 10766 10881 10942 11054 11247 11288 11344 11522
 11595 11722 11732 11835 11839 12054 12061 12644 12727 12733 12922 13006
 13167 13304 13356 13434 13587 13622 13682 13790 13892 14042 14135 14202
 14270 14678 14786 14876 15133 15233 15264 15318 15521 15591 15651 15785
 15799 15815 15840 15883 15990 16000 16035 16328 16336 16360 16381 16394
 16441 16481 16489 16589 16608 16680 16837 17009 17

In [4]:
outliers_exist = df.index.isin(outliers)
df_scaled = df[~outliers_exist]
print('Number of rows after dropping outliers:', df_scaled.shape[0])

Number of rows after dropping outliers: 98079


In [5]:
len(np.where(df['status'].isnull())[0])

0

**Experiment 1**
- Creating out independent and dependent variables
- Using a 5-Fold cross validtion
- Testing our 5-Fold cross-validation on Logisitc Regression and Random Trees

In [6]:
X = df_scaled[["n_hyphens", "path", "url_length", "n_period", "domain_name_correct", "top_domain"]]
y = df_scaled[["status"]]
kfold = KFold(n_splits=5, shuffle=True, random_state=22)

logistic_model = LogisticRegression()

logistic_scores = cross_val_score(logistic_model, X, y, cv=kfold)

random_forest_model = RandomForestClassifier()

random_forest_scores = cross_val_score(random_forest_model, X, y, cv=kfold)

# what scores are these?
print(f'Logistic Regression scores: {logistic_scores}')
print(f'Random Forest scores: {random_forest_scores}')

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Logistic Regression scores: [0.73118883 0.73205546 0.72838499 0.72807912 0.72322202]
Random Forest scores: [0.75810563 0.76554853 0.75917618 0.76269372 0.75870507]


**Experiment 2:**
- Using a Pipeline function to include a Bagging method using a Random Tree
- Using SimpleImputer to replace any missing values with a desciprtive stat if any were missed in preprocessing
- Using 5-fold cross-validation

In [7]:
bag = BaggingClassifier(estimator =RandomForestClassifier(),  n_estimators = 12, oob_score = True, random_state = 22)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
pipeline = Pipeline([('imputer', SimpleImputer()), ('bagging decision tree model', 
                                                    bag)])
accuracy_score_train = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
accuracy_score_test = cross_val_score(pipeline, X_test, y_test, cv=5, scoring='accuracy')
print("Train data accuracy:", accuracy_score_train)
print("Test data accuracy:", accuracy_score_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:791: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:797: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/en

Train data accuracy: [0.76131781 0.76679601 0.7608431  0.7608431  0.76297367]
Test data accuracy: [0.74266296 0.76722274 0.75482775 0.74833926 0.75714506]


**Metrics**
- The following metrics are used to evaluate the performance of our models
    Accuracy, Sensitivity, Specificity, AUC on all three models

In [9]:
kf = KFold(n_splits=5, shuffle=True)
logistic_accuracy_test = []
logistic_sensitivity_test = []
logistic_specificity_test = []
logistic_auc_test = []

logistic_accuracy_train = []
logistic_sensitivity_train = []
logistic_specificity_train = []
logistic_auc_train = []

random_forest_accuracy_test = []
random_forest_sensitivity_test = []
random_forest_specificity_test = []
random_forest_auc_test = []

random_forest_accuracy_train = []
random_forest_sensitivity_train = []
random_forest_specificity_train = []
random_forest_auc_train = []

bag_accuracy_test = []
bag_sensitivity_test = []
bag_specificity_test = []
bag_auc_test = []

bag_accuracy_train = []
bag_sensitivity_train = []
bag_specificity_train = []
bag_auc_train = []

In [11]:
fold_no = 1
for train, test in kf.split(X, y):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    
    logistic_model.fit(X_train, y_train)
    logistic_y_pred_train = logistic_model.predict(X_train) # train
    logistic_y_pred = logistic_model.predict(X_test) # testing
    
    logistic_accuracy_test.append(accuracy_score(y_test, logistic_y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, logistic_y_pred).ravel()
    logistic_sensitivity_test.append(tp / (tp + fn))
    logistic_specificity_test.append(tn / (tn + fp))
    logistic_auc_test.append(roc_auc_score(y_test, logistic_y_pred))
    
    logistic_accuracy_train.append(accuracy_score(y_train, logistic_y_pred_train))
    tn, fp, fn, tp = confusion_matrix(y_train, logistic_y_pred_train).ravel()
    logistic_sensitivity_train.append(tp / (tp + fn))
    logistic_specificity_train.append(tn / (tn + fp))
    logistic_auc_train.append(roc_auc_score(y_train, logistic_y_pred_train))
    
    random_forest_model.fit(X_train, y_train)
    random_forest_y_pred_train = random_forest_model.predict(X_train) # train
    random_forest_y_pred = random_forest_model.predict(X_test) # testing
    
    random_forest_accuracy_test.append(accuracy_score(y_test, random_forest_y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, random_forest_y_pred).ravel()
    random_forest_sensitivity_test.append(tp / (tp + fn))
    random_forest_specificity_test.append(tn / (tn + fp))
    random_forest_auc_test.append(roc_auc_score(y_test, random_forest_y_pred))
    
    random_forest_accuracy_train.append(accuracy_score(y_train, random_forest_y_pred_train))
    tn, fp, fn, tp = confusion_matrix(y_train, random_forest_y_pred_train).ravel()
    random_forest_sensitivity_train.append(tp / (tp + fn))
    random_forest_specificity_train.append(tn / (tn + fp))
    random_forest_auc_train.append(roc_auc_score(y_train, random_forest_y_pred_train))
    
    bag.fit(X_train, y_train)
    bag_y_pred_train = bag.predict(X_train) # train
    bag_y_pred = bag.predict(X_test) # testing
    
    bag_accuracy_test.append(accuracy_score(y_test, bag_y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, bag_y_pred).ravel()
    bag_sensitivity_test.append(tp / (tp + fn))
    bag_specificity_test.append(tn / (tn + fp))
    bag_auc_test.append(roc_auc_score(y_test, bag_y_pred))
    
    bag_accuracy_train.append(accuracy_score(y_train, bag_y_pred_train))
    tn, fp, fn, tp = confusion_matrix(y_train, bag_y_pred_train).ravel()
    bag_sensitivity_train.append(tp / (tp + fn))
    bag_specificity_train.append(tn / (tn + fp))
    bag_auc_train.append(roc_auc_score(y_train, bag_y_pred_train))
    
    print(f"Train Metrics for fold {fold_no}:")
    print(f"Logistic Regression - Accuracy: {logistic_accuracy_train[-1]}, Sensitivity: {logistic_sensitivity_train[-1]}, Specificity: {logistic_specificity_train[-1]}, AUC: {logistic_auc_train[-1]}")
    print(f"Random Forest - Accuracy: {random_forest_accuracy_train[-1]}, Sensitivity: {random_forest_sensitivity_train[-1]}, Specificity: {random_forest_specificity_train[-1]}, AUC: {random_forest_auc_train[-1]}")
    print(f"Bagging Classifier - Accuracy: {bag_accuracy_train[-1]}, Sensitivity: {bag_sensitivity_train[-1]}, Specificity: {bag_specificity_train[-1]}, AUC: {bag_auc_train[-1]}")
    print()
    
    print(f"Test Metrics for fold {fold_no}:")
    print(f"Logistic Regression - Accuracy: {logistic_accuracy_test[-1]}, Sensitivity: {logistic_sensitivity_test[-1]}, Specificity: {logistic_specificity_test[-1]}, AUC: {logistic_auc_test[-1]}")
    print(f"Random Forest - Accuracy: {random_forest_accuracy_test[-1]}, Sensitivity: {random_forest_sensitivity_test[-1]}, Specificity: {random_forest_specificity_test[-1]}, AUC: {random_forest_auc_test[-1]}")
    print(f"Bagging Classifier - Accuracy: {bag_accuracy_test[-1]}, Sensitivity: {bag_sensitivity_test[-1]}, Specificity: {bag_specificity_test[-1]}, AUC: {bag_auc_test[-1]}")
    print()
    
    fold_no += 1

print("Average Testing Metrics:")
print(f"Logistic Regression - Accuracy: {sum(logistic_accuracy_test) / len(logistic_accuracy_test)}, Sensitivity: {sum(logistic_sensitivity_test) / len(logistic_sensitivity_test)}, Specificity: {sum(logistic_specificity_test) / len(logistic_specificity_test)}, AUC: {sum(logistic_auc_test) / len(logistic_auc_test)}")
print(f"Random Forest - Accuracy: {sum(random_forest_accuracy_test) / len(random_forest_accuracy_test)}, Sensitivity: {sum(random_forest_sensitivity_test) / len(random_forest_sensitivity_test)}, Specificity: {sum(random_forest_specificity_test) / len(random_forest_specificity_test)}, AUC: {sum(random_forest_auc_test) / len(random_forest_auc_test)}")
print(f"Bagging Classifier - Accuracy: {sum(bag_accuracy_test) / len(bag_accuracy_test)}, Sensitivity: {sum(bag_sensitivity_test) / len(bag_sensitivity_test)}, Specificity: {sum(bag_specificity_test) / len(bag_specificity_test)}, AUC: {sum(bag_auc_test) / len(bag_auc_test)}")
print(f"OOB Score: {bag.oob_score_}")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_3269475/3178697760.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:791: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/usr/local/lib/

Train Metrics for fold 1:
Logistic Regression - Accuracy: 0.7302295349400354, Sensitivity: 0.8736701469642238, Specificity: 0.5778420291759758, AUC: 0.7257560880700998
Random Forest - Accuracy: 0.7845991104087277, Sensitivity: 0.8763917066653472, Specificity: 0.687081088185044, AUC: 0.7817363974251955
Bagging Classifier - Accuracy: 0.7822158214699922, Sensitivity: 0.8755257558513534, Specificity: 0.6830858194243659, AUC: 0.7793057876378596

Test Metrics for fold 1:
Logistic Regression - Accuracy: 0.7206362153344209, Sensitivity: 0.8624391213597058, Specificity: 0.571323914181057, AUC: 0.7168815177703814
Random Forest - Accuracy: 0.7602977161500816, Sensitivity: 0.8539906569923467, Specificity: 0.6616431187859759, AUC: 0.7578168878891614
Bagging Classifier - Accuracy: 0.7639172104404568, Sensitivity: 0.8579664049299275, Specificity: 0.6648874934589221, AUC: 0.7614269491944247



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/tmp/ipykernel_3269475/3178697760.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_model.fit(X_tr

Train Metrics for fold 2:
Logistic Regression - Accuracy: 0.7286109376393969, Sensitivity: 0.8747217412812268, Specificity: 0.5732916151762942, AUC: 0.7240066782287605
Random Forest - Accuracy: 0.784254999171584, Sensitivity: 0.871852584714321, Specificity: 0.6911366444929403, AUC: 0.7814946146036308
Bagging Classifier - Accuracy: 0.7816295578807846, Sensitivity: 0.8690328963640861, Specificity: 0.6887176925301711, AUC: 0.7788752944471287

Test Metrics for fold 2:
Logistic Regression - Accuracy: 0.7311888254486134, Sensitivity: 0.8762065877201711, Specificity: 0.5788648479147068, AUC: 0.727535717817439
Random Forest - Accuracy: 0.7639172104404568, Sensitivity: 0.8536172753507811, Specificity: 0.6696979199331033, AUC: 0.7616575976419424
Bagging Classifier - Accuracy: 0.7649367862969005, Sensitivity: 0.8537167877400736, Specificity: 0.6716839134524929, AUC: 0.7627003505962833



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_3269475/3178697760.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:791: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/usr/local/lib/

Train Metrics for fold 3:
Logistic Regression - Accuracy: 0.7274001758790768, Sensitivity: 0.8720190188454966, Specificity: 0.5740507326295888, AUC: 0.7230348757375427
Random Forest - Accuracy: 0.7846755795725374, Sensitivity: 0.8703350585671479, Specificity: 0.6938448610892285, AUC: 0.7820899598281882
Bagging Classifier - Accuracy: 0.7819354345360233, Sensitivity: 0.8663232708451994, Specificity: 0.692453127461793, AUC: 0.7793881991534963

Test Metrics for fold 3:
Logistic Regression - Accuracy: 0.7328201468189234, Sensitivity: 0.8718558130322837, Specificity: 0.5853120403446103, AUC: 0.7285839266884468
Random Forest - Accuracy: 0.7636623164763459, Sensitivity: 0.8466032877797584, Specificity: 0.6756671569657491, AUC: 0.7611352223727537
Bagging Classifier - Accuracy: 0.765089722675367, Sensitivity: 0.8461081402257873, Specificity: 0.6791342719058626, AUC: 0.7626212060658251



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_3269475/3178697760.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:791: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/usr/local/lib/

Train Metrics for fold 4:
Logistic Regression - Accuracy: 0.7289040694340008, Sensitivity: 0.8732324608107773, Specificity: 0.5758626122577596, AUC: 0.7245475365342685
Random Forest - Accuracy: 0.7847393038757121, Sensitivity: 0.874916421089126, Specificity: 0.6891182185809569, AUC: 0.7820173198350414
Bagging Classifier - Accuracy: 0.7821011177242777, Sensitivity: 0.8700378891062629, Specificity: 0.6888556273304973, AUC: 0.7794467582183802

Test Metrics for fold 4:
Logistic Regression - Accuracy: 0.7273144371941273, Sensitivity: 0.8754208754208754, Specificity: 0.5701828115150241, AUC: 0.7228018434679498
Random Forest - Accuracy: 0.7603996737357259, Sensitivity: 0.8517528223410576, Specificity: 0.6634797226308048, AUC: 0.7576162724859312
Bagging Classifier - Accuracy: 0.7623368678629691, Sensitivity: 0.8500693206575559, Specificity: 0.6692582475309939, AUC: 0.7596637840942748



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_3269475/3178697760.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:791: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/usr/local/lib/

Train Metrics for fold 5:
Logistic Regression - Accuracy: 0.7275565864600326, Sensitivity: 0.870068972361435, Specificity: 0.5770218564914303, AUC: 0.7235454144264327
Random Forest - Accuracy: 0.7843597063621534, Sensitivity: 0.8765444350717014, Specificity: 0.6869856910739557, AUC: 0.7817650630728284
Bagging Classifier - Accuracy: 0.7818872349102773, Sensitivity: 0.8713094824591873, Specificity: 0.6874312070863253, AUC: 0.7793703447727564

Test Metrics for fold 5:
Logistic Regression - Accuracy: 0.7315829722151415, Sensitivity: 0.8783053179986238, Specificity: 0.5735013768269435, AUC: 0.7259033474127836
Random Forest - Accuracy: 0.7601325516186592, Sensitivity: 0.8526491693699008, Specificity: 0.6604532937936878, AUC: 0.7565512315817943
Bagging Classifier - Accuracy: 0.761916900331379, Sensitivity: 0.8535338641502015, Specificity: 0.6632069476805762, AUC: 0.7583704059153888

Average Testing Metrics:
Logistic Regression - Accuracy: 0.7285101582294403, Sensitivity: 0.8727040875701672, S

In [12]:
train_metrics = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Bagging Classifier'],
    'Accuracy': [sum(logistic_accuracy_train) / len(logistic_accuracy_train), sum(random_forest_accuracy_train) / len(random_forest_accuracy_train), sum(bag_accuracy_train) / len(bag_accuracy_train)],
    'Sensitivity': [sum(logistic_sensitivity_train) / len(logistic_sensitivity_train), sum(random_forest_sensitivity_train) / len(random_forest_sensitivity_train), sum(bag_sensitivity_train) / len(bag_sensitivity_train)],
    'Specificity': [sum(logistic_specificity_train) / len(logistic_specificity_train), sum(random_forest_specificity_train) / len(random_forest_specificity_train), sum(bag_specificity_train) / len(bag_specificity_train)],
    'AUC': [sum(logistic_auc_train) / len(logistic_auc_train), sum(random_forest_auc_train) / len(random_forest_auc_train), sum(bag_auc_train) / len(bag_auc_train)]
})
train_metrics.head()

,Model,Accuracy,Sensitivity,Specificity,AUC
0,Logistic Regression,0.728724,0.873251,0.575368,0.724310
1,Random Forest,0.784523,0.873415,0.690206,0.781811
2,Bagging Classifier,0.781944,0.869870,0.688650,0.779260


In [13]:
test_metrics = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Bagging Classifier'],
    'Accuracy': [sum(logistic_accuracy_test) / len(logistic_accuracy_test), sum(random_forest_accuracy_test) / len(random_forest_accuracy_test), sum(bag_accuracy_test) / len(bag_accuracy_test)],
    'Sensitivity': [sum(logistic_sensitivity_test) / len(logistic_sensitivity_test), sum(random_forest_sensitivity_test) / len(random_forest_sensitivity_test), sum(bag_sensitivity_test) / len(bag_sensitivity_test)],
    'Specificity': [sum(logistic_specificity_test) / len(logistic_specificity_test), sum(random_forest_specificity_test) / len(random_forest_specificity_test), sum(bag_specificity_test) / len(bag_specificity_test)],
    'AUC': [sum(logistic_auc_test) / len(logistic_auc_test), sum(random_forest_auc_test) / len(random_forest_auc_test), sum(bag_auc_test) / len(bag_auc_test)]
})
test_metrics.head()

,Model,Accuracy,Sensitivity,Specificity,AUC
0,Logistic Regression,0.728510,0.872704,0.575917,0.724311
1,Random Forest,0.761570,0.850566,0.667381,0.758973
2,Bagging Classifier,0.763609,0.851179,0.670931,0.761055


**Feature Importances Per Model**

In [14]:
# figure out what the best features are for each model, then print out the top 3 features
logistic_model.fit(X, y)
random_forest_model.fit(X, y)
bag.fit(X, y)

rf_importances = random_forest_model.feature_importances_

lr_importances = np.abs(logistic_model.coef_)
feature_names = df.columns


print("Random Forest:")
for i in np.argsort(rf_importances)[::-1][:3]:
    print(f"{feature_names[i]}: {rf_importances[i]}")


print("Logistic Regression:")
for i in np.argsort(lr_importances)[::-1][:3]:
    print(f"{feature_names[i]}: {lr_importances[0][i]}")
    


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_3269475/1821869398.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_model.fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:804: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Random Forest:
protocol: 0.41955431130439064
www_present: 0.2756030671231525
sub_domain: 0.1784733809503829
Logistic Regression:
Index(['www_present', 'top_domain', 'url', 'domain', 'sub_domain', 'protocol'], dtype='object'): [0.0090159  0.13320302 0.32285809 0.41048689 0.51499088 6.70452586]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:791: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:797: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]
